In [ ]:
https://python.langchain.com/docs/langgraph
https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents

In [ ]:
! pip install langgraph

## Retriever 

In [1]:
import os

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_nomic import NomicEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]



text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=7500, chunk_overlap=100
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

## Retriever Tool

In [18]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Searches and returns information about agents, prompt engineering, and adversarial attacks.",
)

tools = [tool]

We can now wrap these tools in a simple ToolExecutor. 

This is a real simple class that takes in a ToolInvocation and calls that tool, returning the output. 

A ToolInvocation is any class with tool and tool_input attribute.

In [19]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

In [20]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

In [21]:
from langchain.tools.render import format_tool_to_openai_function

functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

## Define the agent state

Graph is parameterized by a state object that it passes around to each node -

* `SET` specific attributes on the state (e.g. overwrite the existing values)
* `ADD` to the existing attribute

State -

* List of messages

We want each node to just add messages to that list - 

* `TypedDict` with one key (messages) 
* Annotate it so that the messages attribute is always added to

In [39]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Nodes

* Function - A function to invoke tools: if the agent decides to take an action, this node will then execute that action.
* Runnable - The agent: responsible for deciding what (if any) actions to take.

## Edges

Some of these edges may be conditional.

The reason they are conditional is that based on the output of a node, one of several paths may be taken.

Conditional edge will take an agent decision - 

* Action - then the function to invoke tools should be called.
* If the agent said that it was finished, then it should finish
  
Normal Edge will take an agent decision - 

* After the tools are invoked, it should always go back to the agent to decide what to do next

In [40]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    print("---DOCS---")
    print(response)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

## Graph

* Start with an agent, `call_model`
* Agent make a decision to call a function
* If so, then `action` to call tool (retriever)
* Then call agent with the tool output added to messages (`state`)

In [42]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

In [43]:
# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [44]:
from langchain_core.messages import HumanMessage
inputs = {"messages": [HumanMessage(content="What are the types of agent memory?")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='What are the types of agent memory?'),
  AIMessage(content="There are several types of agent memory:\n\n1. Episodic Memory: This type of memory allows an agent to remember specific events or episodes that it has experienced. It enables the agent to recall past experiences and use them to inform its current actions.\n\n2. Semantic Memory: Semantic memory refers to the general knowledge and facts that an agent has acquired over time. It includes information about concepts, categories, relationships, and rules. Semantic memory helps the agent understand and interpret the world around it.\n\n3. Procedural Memory: Procedural memory involves the memory of how to perform specific tasks or actions. It includes knowledge of procedures, skills, and routines. Procedural memory allows the agent to execute learned actions without conscious effort.\n\n4. Working Memory: Working memory is a temporary storage system that holds information for a short period of time.

Trace:

https://smith.langchain.com/public/3e22b4f9-9a08-4e64-bab8-47c3843c2418/r